# Введение в анализ данных
## НИУ ВШЭ, 2019-2020 учебный год

### Домашнее задание №3

Задание выполнил(а): Акимова Полина

### Общая информация
Дата выдачи: 

Дедлайн: 

### О задании

В этом домашнем задании вы будете работать с линейной классификацией, попрактикуетесь на реальной задаче классификации текстов.

Для решения этого домашнего задания намного удобнее будет использовать Colab, так как данных много.

### Оценивание и штрафы

За сдачу задания позже срока на итоговую оценку за задание накладывается штраф в размере 1 балл в день, но получить отрицательную оценку нельзя.

__Внимание!__ Домашнее задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов.

### Формат сдачи
Загрузка файлов с решениями происходит в системе Anytask.

### Подготовка данных

In [0]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm

In [90]:
# Датасет можно скачать здесь

!wget https://www.dropbox.com/s/tg55q9mrziroyrs/train_subset.csv

--2020-04-26 11:33:50--  https://www.dropbox.com/s/tg55q9mrziroyrs/train_subset.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.1, 2620:100:6032:1::a27d:5201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/tg55q9mrziroyrs/train_subset.csv [following]
--2020-04-26 11:33:51--  https://www.dropbox.com/s/raw/tg55q9mrziroyrs/train_subset.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc84df927baf2213129e0369f21d.dl.dropboxusercontent.com/cd/0/inline/A2kOkwONc8wGTgDk36BRN8qoedSphc7ZoqzZNbkWAI5Z-psUvgT-KWid8snRsSy1SwRstcFACDEuI5vk0nZE50eAdGx5jYvn8Pid7yQxdBFMXN3Bkr0zeMezWd-91LqlHUU/file# [following]
--2020-04-26 11:33:51--  https://uc84df927baf2213129e0369f21d.dl.dropboxusercontent.com/cd/0/inline/A2kOkwONc8wGTgDk36BRN8qoedSphc7ZoqzZNbkWAI5Z-psUvgT-KWid8snRsSy1SwRstcFACDEuI5vk0nZE50eAdGx5jYvn8Pi

### Данные

Мы имеем дело с данными с торговой платформы Avito.
Для каждого товара представлены следующие параметры:
 - title
 - description
 - Category_name
 - Category

Имеется информация об объектах 50 классов.
Задача: по новым объектам (title, description) предсказать Category.
(Очевидно, что параметр Category_name для предсказания классов использовать нельзя)

In [91]:
data = pd.read_csv("train_subset.csv", index_col='id')

data.head()

,title,description,Category_name,Category
id,,,,
382220,Прихожая,В хорошем состоянии. Торг,Мебель и интерьер,20
397529,Кордиант 215/55/16 Летние,Кордиант 215/55/16 Летние/\n /\nАртикул: 1737l...,Запчасти и аксессуары,10
584569,Стол,"Стол, 2 рабочих места . Стол серого цвета, в д...",Мебель и интерьер,20
2513100,Комбинезон,Размер-42/44,"Одежда, обувь, аксессуары",27
1091886,Ветровка,На 2 года,Детская одежда и обувь,29


In [0]:
data.shape

(30000, 4)

In [0]:
X = data[['title', 'description']].to_numpy()
y = data['Category'].to_numpy()

del data

Сразу разделим выборку на train и test.
Никакие данные из test для обучения использовать нельзя!

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
X_train[:5]

array([['Сапоги 46 размер новые', 'Сапоги 46 размер новые'],
       ['Светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку. В эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iPhone 7 plus 128GB Red красный в наличии',
        '\xa0/\n/\n Данная цена только для подписчиков Instagram: iQmac/\n/\n Новый красный айфон 7 Plus в наличии это элегантный и мощный смартфон, который готов в полной мере раскрыть новые возможности iOS 10. Аппарат с 4-ядерным процессором А10 и 3 ГБ ОЗУ с легкостью решает самые ресурсоемкие задачи, позволяя наслаждаться быстродействием «тяжелых» приложений и игр на 5,5-дюймовом дисплее. Аппарат получил экран, как у iPad Pro, так что картинка теперь соответствует кинематографическому стандарту.'],
       ['Пион Ирис Ромашка рассада',
        'Пион куст 500 р ( более 10 шт)/\nСаженец/ корень 100р/\nРастут у нас более 70 лет/\nРозовые, бордовые и белые/\nНа фото цветы 2018г/\nП. Зубчанинов

### Токенизация (1 балл)


Токенизация -- разбиение текста на мелкие части, которые можно обработать машинными методами.
Можно использовать разные алгоритмы токенизации.
Давайте пока остановимся на простом WordPunctTokenizer.


In [94]:
from nltk.tokenize import WordPunctTokenizer


tokenizer = WordPunctTokenizer()


def preprocess(text: str) -> str:
    return ' '.join(tokenizer.tokenize(text.lower()))


text = 'Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...'
print("before:", text,)
print("after:", preprocess(text),)

before: Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...
after: здраствуйте . я , кирилл . хотел бы чтобы вы сделали игру , 3д - экшон суть такова ...


__Задание:__ Токенизируйте title и description в train и test

In [95]:
# Your code here
for i in range(len(X_train)):
    for j in range(len(X_train[i])):
        X_train[i][j] = preprocess(X_train[i][j])
for i in range(len(X_test)):
    for j in range(len(X_test[i])):
        X_test[i][j] = preprocess(X_test[i][j])
X_train[:10]

array([['сапоги 46 размер новые', 'сапоги 46 размер новые'],
       ['светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку . в эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iphone 7 plus 128gb red красный в наличии',
        '/ / данная цена только для подписчиков instagram : iqmac / / новый красный айфон 7 plus в наличии это элегантный и мощный смартфон , который готов в полной мере раскрыть новые возможности ios 10 . аппарат с 4 - ядерным процессором а10 и 3 гб озу с легкостью решает самые ресурсоемкие задачи , позволяя наслаждаться быстродействием « тяжелых » приложений и игр на 5 , 5 - дюймовом дисплее . аппарат получил экран , как у ipad pro , так что картинка теперь соответствует кинематографическому стандарту .'],
       ['пион ирис ромашка рассада',
        'пион куст 500 р ( более 10 шт )/ саженец / корень 100р / растут у нас более 70 лет / розовые , бордовые и белые / на фото цветы 2018г / п .

In [0]:
X_train[10:11, 1]

array(['продам иж планета 3 , 76 год , ( стоит на старом учёте , документы утеряны ) на ходу , хорошее состояние , все интересующие вопросы по телефону ( с родной коляской на 3 тысячи дороже ) . торга не будет .'],
      dtype=object)

In [0]:
assert X_train[10][1] == 'продам иж планета 3 , 76 год , ( стоит на старом учёте , документы утеряны ) на ходу , хорошее состояние , все интересующие вопросы по телефону ( с родной коляской на 3 тысячи дороже ) . торга не будет .'

### BOW (1.5 балла)

Один из традиционных подходов -- построение bag of words.

Метод состоит в следующем:

 - Составить словарь самых часто встречающихся слов в train data
 - Для каждого примера из train посчитать, сколько раз каждое слово из словаря в нём встречается


 В sklearn есть CountVectorizer, но в этом задании его использовать нельзя.

__Задание:__ Найдите k самых частых слов, отсортируйте их по убыванию частотности

In [0]:
from collections import Counter

list_of_words = []
for i in X_train:
    for j in i:
        list_of_words += map(str, j.split())
counts = Counter(list_of_words)
bow_vocabulary2 = counts.most_common(10000)

In [0]:
result = []
for i in bow_vocabulary2:
    result.append(i[0])
sorted(result)[::200]
assert sorted(result)[::200] == ['!', '12500', '270', '700', 'by', 'gh', 'michael', 'sonata', 'ø', 'аудиоподготовка', 'большим', 'веса', 'воспроизведения', 'габариты', 'гтд', 'джинсами', 'доступность', 'загрузки', 'зимней', 'использовался', 'квартала', 'коммуникации', 'кошки', 'лакированные', 'магазин', 'металл', 'мск', 'натуральным', 'носке', 'одному', 'отвечаем', 'пассат', 'плотно', 'покраску', 'постоянные', 'примеры', 'просьба', 'размещайте', 'репетитор', 'сантехник', 'сидения', 'современного', 'стала', 'схема', 'тон', 'удлиненная', 'фасад', 'цветами', 'шея', 'эту']

In [0]:
def text_to_bow(text: str) -> np.array:
    """
    Возвращает вектор, где для каждого слова из most_common
    указано количество его употреблений
    """ 
    list_of_words = text.split()
    word_vector = []
    count_words = Counter(list_of_words)
    for i in result:
        word_vector.append(count_words[i])
    return np.array(word_vector)
    # Your code here

In [0]:
np.where(np.array(text_to_bow("сдаётся уютный , тёплый гараж для стартапов в ml")) != 0)[0]

array([   1,    4,   12,  565,  866, 1601, 2539, 4063])

In [0]:
assert np.allclose(np.where(text_to_bow("сдаётся уютный , тёплый гараж для стартапов в ml") != 0)[0],
                   np.array([   1,    4,   12,  565,  866, 1601, 2539, 4063]))

In [0]:
def items_to_bow(items: np.array) -> np.array:
    """ Для каждого товара возвращает вектор его bow """
    # Давайте для начала попробуем строить bow только из description товара
    # assert ниже написан для bow из description
    word_matrix = []
    for i in items:
        word_matrix.append(text_to_bow(i[1]))
    return np.array(word_matrix)

    # Your code here

In [0]:
np.where(items_to_bow([X_train[42]])[0] != 0)

(array([   0,    1,    2,    5,    6,    7,   12,   27,   41,   49,  110,
         189,  208,  221, 2032, 3052, 7179, 9568]),)

In [0]:
assert np.allclose(np.where(items_to_bow([X_train[42]])[0] != 0),
                   np.array([   0, 1, 2, 5, 6, 7, 12, 27, 41, 49, 110,
                                189,  208,  221, 2032, 3052, 7179, 9568]))

In [0]:
X_train_bow = items_to_bow(X_train)
X_test_bow = items_to_bow(X_test)

KeyboardInterrupt: ignored

In [0]:
from sklearn.metrics import accuracy_score

### Логистическая регрессия и SVC (1 балл)


Теперь описание каждого товара представлено, как точка в многомерном пространстве.
Очень важно запомнить эту идею: дальше мы будем рассматривать разные способы перехода от текста к точке в пространстве.

Для BOW каждое измерение в пространстве -- какое-то слово.
Мы предполагаем, что текст описывается набором каких-то популярных слов, которые в нём встречаются, а близкие по смыслу тексты будут использовать одинаковые слова.

Обучите логистическую регрессию и SVC с базовыми параметрами.


In [0]:
from sklearn.linear_model import LogisticRegression
bow_model = LogisticRegression(max_iter=100).fit(X_train_bow, y_train)
accuracy_score(bow_model.predict(X_test_bow), y_test)

prediction = bow_model.predict(X_test_bow)
assert accuracy_score(prediction, y_test) > 0.7

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [0]:
accuracy_score(prediction, y_test)

0.7011111111111111

In [0]:
from sklearn.svm import LinearSVC

bow_model = LinearSVC(max_iter=70).fit(X_train_bow, y_train)
print(accuracy_score(bow_model.predict(X_test_bow), y_test))

assert accuracy_score(bow_model.predict(X_test_bow), y_test) > 0.68

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.6847777777777778

### Модификация признаков (0.5 балла)

Добавьте title товара в bow с произвольным весом, как изменится качество?

In [0]:
# Your code here
def all_items_to_bow(items: np.array) -> np.array:
    """ Для каждого товара возвращает вектор его bow """
    # добавляем title к bow с весом 1
    word_matrix = []
    for i in items:
        word_matrix.append(text_to_bow(i[0] + ' ' + i[1]))
    return np.array(word_matrix)

    # Your code here

NameError: ignored

In [0]:
X_train_title_bow = all_items_to_bow(X_train)

In [0]:
X_test_title_bow = all_items_to_bow(X_test)

In [0]:
from sklearn.linear_model import LogisticRegression
bow_model = LogisticRegression(max_iter=100).fit(X_train_title_bow, y_train)
accuracy_score(bow_model.predict(X_test_title_bow), y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7812222222222223

In [0]:
from sklearn.svm import LinearSVC

bow_model = LinearSVC(max_iter=70).fit(X_train_title_bow, y_train)
accuracy_score(bow_model.predict(X_test_title_bow), y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.7553333333333333

Нормализуйте данные (`sklearn.preprocessing.normalize`) перед обучением. Что станет с качеством и почему?

In [0]:
# Your code here
from sklearn.preprocessing import normalize
X_train_norm = normalize(X_train_title_bow)
X_test_norm = normalize(X_test_title_bow)
del X_train_title_bow
del X_test_title_bow

In [0]:
from sklearn.linear_model import LogisticRegression
bow_model = LogisticRegression(max_iter=100).fit(X_train_norm, y_train)
accuracy_score(bow_model.predict(X_test_norm), y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.677

In [0]:
from sklearn.svm import LinearSVC

bow_model = LinearSVC(max_iter=70).fit(X_train_norm, y_train)
accuracy_score(bow_model.predict(X_test_norm), y_test)

0.7987777777777778

In [ ]:
# качество на логистической регрессии понизилось после нормализации, на методе опорных векторов повысилось.

### mystem (0.5) балла

Попробуйте обучиться, используя токенизатор mystem. Сравните качество.

In [0]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /bin

In [0]:
pip install git+https://github.com/nlpub/pymystem3

In [0]:
from pymystem3 import Mystem

# Your code here

### TF-IDF (1.5 балла)

Не все слова полезны одинаково, давайте попробуем [взвесить](http://tfidf.com/) их, чтобы отобрать более полезные.


> TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).
> 
> IDF(t) = log_e(Total number of documents / Number of documents with term t in it).


В sklearn есть TfidfVectorizer, но в этом задании его использовать нельзя.

In [0]:
# Давайте для простоты считать один tf-idf для title и description.
# Для каждого слова из bow_vocabulary нужно посчитать
# в тексте скольких товаров встретилось это слово


word_vector = np.zeros(len(result))
for i in range(len(result)):
    for j in X_train:
        if result[i] in j[0] or result[i] in j[1]:
            word_vector[i] += 1

# count_arr = #
word_vector[:200]

array([10383., 14742., 16346.,  9671., 19347., 20154., 15227.,  5697.,
        4903., 19681.,  5609., 13071.,  5535., 10044.,  5859., 10399.,
       11073.,  4385.,  9293., 12603.,   675.,  1826.,  7931.,  8600.,
        3818.,  2122., 17098.,  9398.,  3928.,  3767.,  3234.,  5638.,
         757.,  7620.,  2352.,  2080.,  6385.,  4297., 19587.,  2434.,
        2584.,  3348.,  1698.,  4502.,  2266.,  1507., 15855.,   211.,
        1815.,  2046.,  6555.,  4764.,  1798., 20123.,  1822.,  1641.,
        1801.,  2126., 19018.,  1140.,   368.,  1231.,  1466.,  6054.,
        1547.,  4889.,  5253.,  1556.,  1454.,  1306.,  1460.,  5747.,
        5062.,   859., 20492.,   959.,  2630.,   543.,  2885., 14423.,
         661.,   715.,  1433.,  1045.,  1980.,  1613.,   667.,  1124.,
        9144.,  1700.,  1907.,  2075.,  1111.,  1086.,  3377.,    75.,
        1792.,   990.,   938.,  1070.,  1863.,  1193.,   901.,  1006.,
        2041., 11012.,  2127.,  1093.,  1157.,  1521.,   848.,   865.,
      

In [0]:
def text_to_tfidf(text: str) -> np.array:
    """
    Возвращает вектор, где для каждого слова из most_common
    указан tf-idf
    """
    list_of_words = text.split()
    dct_words = Counter(list_of_words)
    tf_idf_vector = []
    for i in range(len(result)):
        tf = dct_words[result[i]] / len(list_of_words)
        idf = np.log(len(X_train) / word_vector[i])
        tf_idf_vector.append(tf*idf)
    return tf_idf_vector
    # Your code here

In [0]:
tf_idf_features = []
for i in X_train:
    tf_idf_features += text_to_tfidf(i[0] + ' ' + i[1])
tf_idf_ft = np.array(tf_idf_features).reshape((len(tf_idf_features) // len(result), len(result)))

In [0]:
# Нормализуйте данные
from sklearn.preprocessing import normalize
X_train_tfidf_norm = normalize(tf_idf_ft)

In [0]:
del tf_idf_features
del tf_idf_ft

In [0]:
tf_idf_features = []
for i in X_test:
    tf_idf_features += text_to_tfidf(i[0] + ' ' + i[1])
tf_idf_ft = np.array(tf_idf_features).reshape((len(tf_idf_features) // len(result), len(result)))

In [0]:
# Нормализуйте данные
X_test_tfidf_norm = normalize(tf_idf_ft)
# Your code here

In [0]:
del tf_idf_features
del tf_idf_ft

### Модели на TF-IDF признаках (1 балл)

Обучите логистическую регрессию и SVC, оцените качество (accuracy_score)

In [0]:
# Your code here
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
tfidf_model = LogisticRegression(max_iter=100).fit(X_train_tfidf_norm, y_train)
accuracy_score(tfidf_model.predict(X_test_tfidf_norm), y_test)

# /usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
# STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

# Increase the number of iterations (max_iter) or scale the data as shown in:
#     https://scikit-learn.org/stable/modules/preprocessing.html
# Please also refer to the documentation for alternative solver options:
#     https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
#   extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)

# 0.7643333333333333

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7643333333333333

In [0]:
from sklearn.svm import LinearSVC

tfidf_model = LinearSVC(max_iter=70).fit(X_train_tfidf_norm, y_train)
accuracy_score(tfidf_model.predict(X_test_tfidf_norm), y_test)

# 0.8057777777777778

0.8057777777777778

### Hashing Vectorizer (0.5 балла)

Попробуйте использовать `sklearn.feature_extraction.text.HashingVectorizer` для векторизации текстов.
Обязательно оцените качество работы алгоритмов классификации с использованием новой векторизации.

In [0]:
def x_for_hv(items: np.array) -> np.array:
    list_for_hv = []
    for i in items:
        list_for_hv.append(i[0] + ' ' + i[1])
    return list_for_hv

In [0]:
X_train_for_hv = x_for_hv(X_train)
X_test_for_hv = x_for_hv(X_test)

In [0]:
# Your code here
from sklearn.feature_extraction.text import HashingVectorizer
vectorizer = HashingVectorizer(n_features=2**14)
X_train_hv = vectorizer.fit_transform(X_train_for_hv)
print(X_train_hv.shape)

(21000, 16384)


In [0]:
vectorizer = HashingVectorizer(n_features=2**14)
X_test_hv = vectorizer.fit_transform(X_test_for_hv)
print(X_test_hv.shape)

(9000, 16384)


In [0]:
hv_model = LogisticRegression(max_iter=100).fit(X_train_hv, y_train)
accuracy_score(hv_model.predict(X_test_hv), y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7277777777777777

In [0]:
hv_model = LinearSVC(max_iter=70).fit(X_train_hv, y_train)
accuracy_score(hv_model.predict(X_test_hv), y_test)

0.8076666666666666

### Word Vectors (1 балл)

Давайте попробуем другой подход -- кажому слову сопоставим какой-то эмбеддинг (вектор).

Вектора будут небольшой размерности. Таким образом мы снизим количество параметров в модели.

Вектора мы возьмём уже готовые (обученные на текстах их интернета), так что наша модель будет знать некоторую дополнительную информацию о внешнем мире.

In [0]:
!wget https://www.dropbox.com/s/0x7oxso6x93efzj/ru.tar.gz

--2020-04-15 15:22:33--  https://www.dropbox.com/s/0x7oxso6x93efzj/ru.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.1, 2620:100:6032:1::a27d:5201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/0x7oxso6x93efzj/ru.tar.gz [following]
--2020-04-15 15:22:34--  https://www.dropbox.com/s/raw/0x7oxso6x93efzj/ru.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1f9f3824b5d9dd950ccb4f16fa.dl.dropboxusercontent.com/cd/0/inline/A14dmMZHW5K0ZKZXQ1Hl0ul20vpSkqrbmrC0P14VBznFVYTS---2BUuCgateNJH56VSEmvuwgiuLyVg9KM7rerUWPMTcsBXYI8u6L9Qqsjv4mA/file# [following]
--2020-04-15 15:22:34--  https://uc1f9f3824b5d9dd950ccb4f16fa.dl.dropboxusercontent.com/cd/0/inline/A14dmMZHW5K0ZKZXQ1Hl0ul20vpSkqrbmrC0P14VBznFVYTS---2BUuCgateNJH56VSEmvuwgiuLyVg9KM7rerUWPMTcsBXYI8u6L9Qqsjv4mA/file
Resolving uc1f9f3824b5d9d

In [0]:
!tar -xzvf ru.tar.gz

ru.bin
ru.vec


In [0]:
import gensim
from gensim.models.wrappers import FastText


model = FastText.load_fasttext_format('ru.bin')

In [0]:
# Эмбеддинг предложения -- сумма эмбеддингов токенов


def sentence_embedding(sentence: str) -> np.array:
    """
    Складывает вектора токенов строки sentence
    """

    embedding_dim = model['кек'].shape[0]
    features = np.zeros([embedding_dim], dtype='float32')
    
    for word in sentence.split():
        if word in model:
            features += model[word]
    
    return features

In [0]:
assert np.allclose(sentence_embedding('сдаётся уютный , тёплый гараж для стартапов в ml')[::50],
                   np.array([ 0.08189847,  0.07249198, -0.15601222,  0.03782297,  0.09215296, -0.23092946]))

In [0]:
train_items_embedding = []
for i in X_train:
    train_items_embedding.append(sentence_embedding(i[0] + ' ' + i[1]))
train_items_emb = np.array(train_items_embedding)

test_items_embedding = []
for i in X_test:
    test_items_embedding.append(sentence_embedding(i[0] + ' ' + i[1]))
test_items_emb = np.array(test_items_embedding)


In [0]:
emb_model = LogisticRegression(max_iter=100).fit(train_items_emb, y_train)
accuracy_score(emb_model.predict(test_items_emb), y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.5548888888888889

In [0]:
emb_model = LinearSVC(max_iter=70).fit(train_items_emb, y_train)
accuracy_score(emb_model.predict(test_items_emb), y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.5333333333333333

In [0]:
# Обучите логистическую регрессию и SVM
# Оцените качество (accuracy_score)

### Что дальше?

Решение каждого пункта 1 балл:

1. N-Gram модели текстовой классификации

2. Обучиться на полных данных (контест на kaggle)

3. Поработать с другими эмбеддингами (word2vec, GloVe).

4. Использовать Vowpal Wabbit вместо sklearn.

5. Другие способы токенизации (pymorphy2, spaCy)


Снабжайте код пояснениями и графиками.
Обязательно необходимо написать вывод по каждому пункту, который вы реализуете.

In [96]:
from gensim.models import word2vec

train_for_w2v = []
for i in X_train:
    train_for_w2v.append(i[0] + i[1]) # создаем тренировочную выборку для модели word2vec из title и description

w2v = word2vec.Word2Vec(train_for_w2v, size=300, window=3, workers=4) # обучаем модель на тренировочной выборке, длина векторов = 300
model = dict(zip(w2v.wv.index2word, w2v.wv.syn0)) # создаем словарь, где слову сопоставлен вектор, из полученной выше модели

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  


In [0]:
del train_for_w2v

In [0]:
def w2v_embedding(sentence: str) -> np.array:
    """
    Складывает вектора токенов строки sentence
    """

    features = np.zeros([300], dtype='float32')
    
    for word in sentence.split():
        if word in model:
            features += model[word]
    
    return features # аналогично FastText, получаем эмбеддинг предложения

In [0]:
# преобразуем каждое предложение в вектор
X_train_w2v = []
for i in X_train:
    X_train_w2v.append(w2v_embedding(i[0] + ' ' + i[1]))
X_train_word2vec = np.array(X_train_w2v)

X_test_w2v = []
for i in X_test:
    X_test_w2v.append(w2v_embedding(i[0] + ' ' + i[1]))
X_test_word2vec = np.array(X_test_w2v)

In [0]:
X_train_word2vec.shape

(21000, 300)

In [99]:
# обучаем логистическую регрессию, смотрим результаты
from sklearn.linear_model import LogisticRegression
w2v_model = LogisticRegression(max_iter=100).fit(X_train_word2vec, y_train)
accuracy_score(w2v_model.predict(X_test_word2vec), y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.3042222222222222

In [100]:
# обучаем LinearSVC, смотрим результаты
from sklearn.svm import LinearSVC

w2v_model = LinearSVC(max_iter=70).fit(X_train_word2vec, y_train)
accuracy_score(w2v_model.predict(X_test_word2vec), y_test)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.14466666666666667

In [0]:
# наблюдается заметное снижение качества. Возможно, увеличение количества итераций при обучении, удаление стоп-слов улучшило бы качество. Сейчас алгоритм word2vec не находит взаимосвязи между словами.

In [0]:
from collections import defaultdict

model = defaultdict(lambda: defaultdict(lambda: 0))

# составим словарь триграмм, ключи словаря - пары последовательных слов в предложении, значения - словари, 
# в которых ключи это слова, идущие после биграмм (третьи слова в триграмме), значения - количество триграмм, образованных этими тремя словами. 
for sentence in np.hstack((X_train[:, 0], X_train[:, 1])):
    sentence_split = [i for i in sentence.split()]
    for i in range(len(sentence_split) - 2):
        model[(sentence_split[i], sentence_split[i + 1])][sentence_split[i + 2]] += 1
  

In [0]:
# заменим значения в словарях с количества триграмм в тексте на долю триграмм от всех триграмм, имеющих аналогичное начало
for w1_w2 in model:
    total_count = float(sum(model[w1_w2].values()))
    for w3 in model[w1_w2]:
        model[w1_w2][w3] /= total_count
# получили NGram модель    

In [38]:
model['сапоги', '46']

defaultdict(<function __main__.<lambda>.<locals>.<lambda>>, {'размер': 1.0})

In [0]:
# получаем probability для title и description каждого объекта тестовой выборки
result_probabilities = []
for i in X_test:
    title_prob = 1
    descr_prob = 1
    for j in range(len(i)):
        sentence_split = [k for k in i[j].split()]
        for k in range(len(sentence_split) - 2):
            if (sentence_split[k], sentence_split[k + 1]) in model:
                if sentence_split[k + 2] in model[sentence_split[k], sentence_split[k + 1]]:
                    if j == 0:
                        title_prob *= model[sentence_split[k], sentence_split[k + 1]][sentence_split[k + 2]]
                    else:
                        descr_prob *= model[sentence_split[k], sentence_split[k + 1]][sentence_split[k + 2]]
    result_probabilities.append([title_prob, descr_prob])

In [40]:
result_probabilities[2]

[0.038461538461538464, 6.496309225032204e-28]

In [41]:
# другие способы токенизации - pymorphy2 и spaCy
pip install pymorphy2

     |████████████████████████████████| 51kB 1.6MB/s 
     |████████████████████████████████| 7.1MB 6.9MB/s 


In [0]:
from pymorphy2.tokenizers import simple_word_tokenize

# функция для токенизации предложения с помощью pymorphy2
def preprocess_pm(text: str) -> str:
    return ' '.join(simple_word_tokenize(text.lower()))


In [0]:
new_data = pd.read_csv("train_subset.csv", index_col='id')
X = new_data[['title', 'description']].to_numpy()
y = new_data['Category'].to_numpy()

del new_data

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [49]:
for i in range(len(X_train)):
    for j in range(len(X_train[i])):
        X_train[i][j] = preprocess_pm(X_train[i][j])
for i in range(len(X_test)):
    for j in range(len(X_test[i])):
        X_test[i][j] = preprocess_pm(X_test[i][j])
X_train[:10]
# токенизовали все предложения в тренировочной и тестовой выборках

array([['сапоги 46 размер новые', 'сапоги 46 размер новые'],
       ['светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку . в эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iphone 7 plus 128gb red красный в наличии',
        '/ / данная цена только для подписчиков instagram : iqmac / / новый красный айфон 7 plus в наличии это элегантный и мощный смартфон , который готов в полной мере раскрыть новые возможности ios 10 . аппарат с 4-ядерным процессором а10 и 3 гб озу с легкостью решает самые ресурсоемкие задачи , позволяя наслаждаться быстродействием « тяжелых » приложений и игр на 5 , 5-дюймовом дисплее . аппарат получил экран , как у ipad pro , так что картинка теперь соответствует кинематографическому стандарту .'],
       ['пион ирис ромашка рассада',
        'пион куст 500 р ( более 10 шт ) / саженец / корень 100р / растут у нас более 70 лет / розовые , бордовые и белые / на фото цветы 2018г / п . зу

In [0]:
list_of_words = []
for i in X_train:
    for j in i:
        list_of_words += map(str, j.split())
counts = Counter(list_of_words)
bow_vocabulary2 = counts.most_common(10000)

In [0]:
result = []
for i in bow_vocabulary2:
    result.append(i[0])
# составли список самых частовстречающихся слов в выборке

In [0]:
X_train_bow_pm = items_to_bow(X_train)
X_test_bow_pm = items_to_bow(X_test)
# каждый объект выборок представляем в виде вектора способом bag-of-words

In [57]:
from sklearn.linear_model import LogisticRegression
pm_bow_model = LogisticRegression(max_iter=100).fit(X_train_bow_pm, y_train)
accuracy_score(pm_bow_model.predict(X_test_bow_pm), y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.6472222222222223

In [58]:
from sklearn.svm import LinearSVC

pm_bow_model = LinearSVC(max_iter=70).fit(X_train_bow_pm, y_train)
accuracy_score(pm_bow_model.predict(X_test_bow_pm), y_test)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.6824444444444444

In [0]:
# после обучения оценили качество. Оно ниже чем при токенизации в первом задании в случае с
# логистической регрессией и близко к качеству на методе опорных векторов.

In [0]:
del X_test_bow_pm
del X_train_bow_pm

In [59]:
pip install git+https://github.com/aatimofeev/spacy_russian_tokenizer.git

  Cloning https://github.com/aatimofeev/spacy_russian_tokenizer.git to /tmp/pip-req-build-mucpi5vb
  Running command git clone -q https://github.com/aatimofeev/spacy_russian_tokenizer.git /tmp/pip-req-build-mucpi5vb
  Created wheel for spacy-russian-tokenizer: filename=spacy_russian_tokenizer-0.1.1-cp36-none-any.whl size=12675 sha256=34271d77cd72fb85e284ad5b127addd76df05e01452c6bb19e8bf6d8856fb91e
  Stored in directory: /tmp/pip-ephem-wheel-cache-u7iy5f1q/wheels/37/3b/bb/cfe712f7c0b78cd08f4a2ef122d17748baf9d4bebecf2e5a54
Successfully built spacy-russian-tokenizer


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [0]:
from spacy.lang.ru import Russian
# используем библиотеку для токенизации русскоязычного текста

def preprocess_spacy(text: str) -> str:
    nlp = Russian()
    return ' '.join([token.text for token in nlp(text)])


In [65]:
preprocess_spacy('сегодня, хорошая! погода')

'сегодня , хорошая ! погода'

In [0]:
for i in range(len(X_train)):
    for j in range(len(X_train[i])):
        X_train[i][j] = preprocess_spacy(X_train[i][j])
# токенизовали все предложения тренировочной выборки

In [75]:
X_train[:10]

array([['сапоги 46 размер новые', 'сапоги 46 размер новые'],
       ['светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку . в эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iphone 7 plus 128 gb red красный в наличии',
        '\xa0      / \n      / \n       данная цена только для подписчиков instagram : iqmac/ \n      / \n       новый красный айфон 7 plus в наличии это элегантный и мощный смартфон , который готов в полной мере раскрыть новые возможности ios 10 . аппарат с 4-ядерным процессором а10 и 3 гб озу с легкостью решает самые ресурсоемкие задачи , позволяя наслаждаться быстродействием « тяжелых » приложений и игр на 5,5-дюймовом дисплее . аппарат получил экран , как у ipad pro , так что картинка теперь соответствует кинематографическому стандарту .'],
       ['пион ирис ромашка рассада',
        'пион куст 500 р ( более 10 шт)/ \n      саженец/ корень 100р/ \n      растут у нас более 70 лет/ \n  

In [0]:
for i in range(len(X_test)):
    for j in range(len(X_test[i])):
        X_test[i][j] = preprocess_spacy(X_test[i][j])
# токенизовали все предложения тестовой выборки

In [73]:
X_train.shape

(21000, 2)

In [0]:
list_of_words = []
for i in X_train:
    for j in i:
        list_of_words += map(str, j.split())
counts = Counter(list_of_words)
bow_vocabulary2 = counts.most_common(10000)

In [0]:
result = []
for i in bow_vocabulary2:
    result.append(i[0])
# получили список самых частовстречающихся слов

In [0]:
X_train_bow_spacy = items_to_bow(X_train)
X_test_bow_spacy = items_to_bow(X_test)
# векторизовали каждый объект в выборках методом bag-of-words

In [87]:
spacy_bow_model = LogisticRegression(max_iter=100).fit(X_train_bow_spacy, y_train)
accuracy_score(spacy_bow_model.predict(X_test_bow_spacy), y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.6174444444444445

In [88]:
spacy_bow_model = LinearSVC(max_iter=70).fit(X_train_bow_spacy, y_train)
accuracy_score(spacy_bow_model.predict(X_test_bow_spacy), y_test)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.5912222222222222

In [0]:
# при таком способе токенизации наблюдается снижение качества. Возможно, что после токенизации в выборке остались ненужные слова или наборы символов. 
# В каждом примере можно также увеличить количество итераций, но тогда обучение и предсказание будут строиться дольше.